<a href="https://colab.research.google.com/github/kgudalewski/trading/blob/main/trading_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# TODO
# sprawdzić dostępność danych dla rynku kryptowalut
# strategia
# proste metody oceny wyników strategii

#offtop
# poszukać źródeł danych które by zawierały minutowe timeframey

In [2]:
real_trade = True

# Set up

In [3]:
# set up
if real_trade:
  import MetaTrader5 as mt5
  from datetime import datetime, timedelta, timezone
  import time
  import pandas as pd
  import pytz
elif not real_trade:
  from datetime import datetime, timedelta
  import time
  import pandas as pd
  from google.colab import drive
  drive.mount('/content/drive')
  import matplotlib.pyplot as plt
  from sklearn.preprocessing import StandardScaler
  import pytz

# Functions

In [4]:
# MT5 connection init function
if real_trade:
  def initialize_mt5(login, password, server):

      if not mt5.initialize(login=login, password=password, server=server):
          print(f"Nie można połączyć się z MetaTrader 5, błąd: {mt5.last_error()}")
          return False
      print("Połączono z MetaTrader 5")

  # if doesn't work
  # mt5.initialize()
  # mt5.login(login, password, server)

In [5]:
# order send func

if real_trade:

  def create_order(order_type, idx=None, ticker=None, qty=None, sl_pct=None , tp_pct=None):

    if order_type == "BUY":
      order_type = mt5.ORDER_TYPE_BUY
      price = mt5.symbol_info_tick(ticker).ask
      sl = price * (1  - sl_pct)
      tp = price * (1 + tp_pct)
    elif order_type == "SELL":
      order_type = mt5.ORDER_TYPE_SELL
      price = mt5.symbol_info_tick(ticker).bid
      sl = price * (1  + sl_pct)
      tp = price * (1 - tp_pct)

    request = {
      "action": mt5.TRADE_ACTION_DEAL,
      "symbol": ticker,
      "volume": qty,
      "type": order_type,
      "price": price,
      "sl": sl, # FLOAT
      "tp": tp, # FLOAT,
      "comment": "Python open position",
      "type_time": mt5.ORDER_TIME_GTC,
      "type_filling": mt5.ORDER_FILLING_IOC,
    }

    order = mt5.order_send(request)
    return order

elif not real_trade:

  def create_order(order_type, idx):
    if order_type == "BUY": # to znaczy otwarcie longa
      source_df.loc[idx,"open_long"] = 1 # w ostatnim wierszu , kolumnie open_long sygnalizuje otwarcie longa "1"ką
    if order_type == "SELL": # to znaczy otwarcie shorta
      source_df.loc[idx,"open_short"] = 1 # w ostatnim wierszu , kolumnie open_short sygnalizuje otwarcie shorta "1"ką

In [6]:
#order close func

if real_trade:

  def close_order(order_type, idx=None, ticker=None, qty=None):

    if order_type == "BUY":
      order_type = mt5.ORDER_TYPE_BUY
      price = mt5.symbol_info_tick(ticker).ask
    elif order_type == "SELL":
      order_type = mt5.ORDER_TYPE_SELL
      price = mt5.symbol_info_tick(ticker).bid

    request = {
      "action": mt5.TRADE_ACTION_DEAL,
      "symbol": ticker,
      "volume": qty,
      "type": order_type,
      "position": mt5.positions_get()[0]._asdict()['ticket'],
      "price": price,
      "comment": "Close position",
      "type_time": mt5.ORDER_TIME_GTC,
      "type_filling": mt5.ORDER_FILLING_IOC,
    }
    order = mt5.order_send(request)
    return order

elif not real_trade:

  def close_order(order_type, idx):
    if order_type == "BUY": # zamkniecie shorta poprzez wykupienie go
      source_df.loc[idx,"close_short"] = 1 # w ostatnim wierszu , kolumnie close_short sygnalizuje zamkniecie shorta "1"ką (poprzez wykupienie go)
    if order_type == "SELL": # zamkniecie longa poprzez sprzedanie go
      source_df.loc[idx,"close_long"] = 1 # w ostatnim wierszu , kolumnie close_long sygnalizuje zamkniecie longa "1"ką (poprzez sprzedanie go)

In [7]:
# preparing df for imitation

if not real_trade:

  def prepare_for_imitation(df):

    df["open_short"] = 0
    df["close_short"] = 0
    df["open_long"] = 0
    df["close_long"] = 0
    return df

# Introduction to strategy

In [8]:
if real_trade:
  login = 1520661470
  password = "@6$Ejp!k"
  server = "FTMO-Demo2"

  initialize_mt5(login, password, server)

Połączono z MetaTrader 5


In [9]:
# basic info

ticker = "EURUSD"
sl_pct = .05
tp_pct = .1
qty = 1.0
# interval = mt5.TIMEFRAME_D1
# from_date = datetime.now()
# no_of_rows = 100
# rates = mt5.copy_rates_from(ticker, interval, from_date, no_of_rows)
# account_info = mt5.account_info()
# account_info.balance
# account_info.login
# num_symbols = mt5.symbols_total()
# symbol_info = mt5.symbols_get()
# mt5.symbol_info(ticker)._asdict()

In [10]:
if not real_trade:
  source_df = prepare_for_imitation(pd.read_csv("/content/drive/MyDrive/TSLA_M30_01_2021-11_2024.csv", index_col=0))
  display(source_df.iloc[:10])

# Strategy

In [11]:
if real_trade:
  [(position._asdict()['ticket'],position._asdict()['symbol']) for position in list(mt5.positions_get())]

In [12]:
# needed for strategy execution, depending on real_trade condition
if real_trade:
  iterations = range(600)
elif not real_trade:
  reach = 5
  iterations = range(reach, source_df.shape[0])

In [16]:
# if not real_trade, this conditions need to be predeclared
if not real_trade:
  already_buy = False
  already_sell = False
  no_position = True

# loop
for i in iterations:

  if real_trade:
    df = pd.DataFrame(mt5.copy_rates_range(ticker, mt5.TIMEFRAME_M1, datetime.now() - timedelta(days=7), datetime.now()+timedelta(hours=2))) # updates df each iteration
  elif not real_trade:
    df = source_df.iloc[i-5:i].copy() # updates df each iteration
  # df['time'] = pd.to_datetime(df['time'], unit = 's') # nie wiem czy to jest potrzebne tutaj do czegoś

# --------------------------------------------------------------------- #

# getting info
  current_close = df.iloc[-1]["close"] # aktualna cena zamknięcia
  last_close = df.iloc[-2]["close"] # wiersz wcześniej cena zamknięcia
  last_high = df.iloc[-2]["high"] # wiersz wcześniej cena high
  last_low = df.iloc[-2]["low"] # wiersz wcześniej cena low
  print(f"Current date : {pd.to_datetime(df.iloc[-1]['time'], unit = 's')}")
  print(f"current_close : {current_close}")
  print(f"last_close : {last_close}")
  print(f"last_high : {last_high}")
  print(f"last_low : {last_low}")
    
  # strategy with output to the rest of code with signal what to do
  long_condition = current_close > last_high
  short_condition = current_close < last_low
  closelong_condition = current_close < last_close
  closeshort_condition = current_close > last_close
  print(f"long_condition : {long_condition}")
  print(f"short_condition : {short_condition}")
  print(f"closelong_condition : {closelong_condition}")
  print(f"closeshort_condition : {closeshort_condition}")

# --------------------------------------------------------------------- #

  # downloading data about current positions from mt5, if real_trade condition met
  if real_trade:
    already_buy = False
    already_sell = False
    try:
      already_sell = mt5.positions_get()[0]._asdict()['type'] == 1
      already_buy = mt5.positions_get()[0]._asdict()['type'] == 0
    except:
      pass

    no_position = len(mt5.positions_get()) == 0

  if long_condition:
    if no_position:
      if real_trade:
        create_order(ticker=ticker, qty=qty, order_type="BUY", sl_pct = sl_pct , tp_pct = tp_pct)
        print("Buy order placed, due to long_condition")
      elif not real_trade:
        create_order("BUY",i)
        long_buy_price = df.iloc[-1]['close']
        no_position = False
        already_buy = True
        # print("Buy order placed, due to long_condition")
    if already_sell:
      if real_trade:
        close_order(ticker=ticker, qty=qty, order_type="BUY")
        print("Sell position closed, due to long_condition")
        time.sleep(1)
        create_order(ticker=ticker, qty=qty, order_type="BUY", sl_pct = sl_pct , tp_pct = tp_pct)
        print("Buy order placed, due to long_condition")
      elif not real_trade:
        close_order("BUY",i)
        # print("Sell position closed, due to long_condition")
        no_position = True
        already_sell = False
        create_order("BUY", i)
        # print("Buy order placed, due to long_condition")
        long_buy_price = df.iloc[-1]['close']
        no_position = False
        already_buy = True
  if short_condition:
    if no_position:
      if real_trade:
        create_order(ticker=ticker, qty=qty, order_type="SELL",sl_pct = sl_pct , tp_pct = tp_pct)
        print("Sell order placed, due to short_condition")
      elif not real_trade:
        create_order("SELL", i)
        # print("Sell order placed, due to short_condition")
        short_sell_price = df.iloc[-1]['close']
        no_position = False
        already_sell = True
    if already_buy:
      if real_trade:
        close_order(ticker=ticker, qty=qty, order_type="SELL")
        print("Buy position closed, due to short_condition")
        time.sleep(1)
        create_order(ticker=ticker, qty=qty, order_type="SELL",sl_pct = sl_pct , tp_pct = tp_pct)
        print("Sell order placed, due to short_condition")
      elif not real_trade:
        close_order("SELL", i)
        # print("Buy position closed, due to short_condition")
        no_position = True
        already_buy = False
        create_order("SELL", i)
        # print("Sell order placed, due to short_condition")
        short_sell_price = df.iloc[-1]['close']
        no_position = False
        already_sell = True

  if not real_trade:
    if already_buy:
      long_sell_price = df.iloc[-1]['close']
      if long_sell_price/long_buy_price > (1 + tp_pct) or long_sell_price/long_buy_price < (1 - sl_pct) : # take profit and stop loss for buy order
        close_order("SELL", i)
        already_buy = False
        no_position = True

    if already_sell:
      short_buy_price = df.iloc[-1]['close']
      if short_sell_price/short_buy_price > (1 + tp_pct) or short_sell_price/short_buy_price < (1 - sl_pct) : # take profit and stop loss for sell order
        close_order("BUY", i)
        already_sell = False
        no_position = True


  if real_trade:
    already_buy = False
    already_sell = False
      
  if real_trade:
    try:
      already_sell = mt5.positions_get()[0]._asdict()['type'] == 1
      already_buy = mt5.positions_get()[0]._asdict()['type'] == 0
    except:
      pass

  if closelong_condition and already_buy:
      if real_trade:
        close_order(ticker=ticker, qty=qty, order_type="SELL")
        print("Only buy position closed")
      elif not real_trade:
        close_order("SELL", i)
        already_buy = False
        no_position = True
  if closeshort_condition and already_sell:
      if real_trade:
        close_order(ticker=ticker, qty=qty, order_type="BUY")
        print("Only sell position closed")
      elif not real_trade:
        close_order("BUY", i)
        already_sell = False
        no_position = True

  # if real_trade:
  #   already_buy = False
  #   already_sell = False

  if real_trade:
      print(f"already_buy: {already_buy}")
      print(f"already_sell: {already_sell}")
      print("-----------------------------------------------------------------------")
      time.sleep(60)

Current date : 2025-02-06 08:15:00
current_close : 1.03862
last_close : 1.0386
last_high : 1.03861
last_low : 1.03858
long_condition : True
short_condition : False
closelong_condition : False
closeshort_condition : True
Buy order placed, due to long_condition
already_buy: True
already_sell: False
-----------------------------------------------------------------------
Current date : 2025-02-06 08:16:00
current_close : 1.03866
last_close : 1.03865
last_high : 1.03868
last_low : 1.0386
long_condition : False
short_condition : False
closelong_condition : False
closeshort_condition : True
already_buy: True
already_sell: False
-----------------------------------------------------------------------
Current date : 2025-02-06 08:17:00
current_close : 1.03866
last_close : 1.03865
last_high : 1.03868
last_low : 1.03862
long_condition : False
short_condition : False
closelong_condition : False
closeshort_condition : True
already_buy: True
already_sell: False
---------------------------------------

# Backtesting

In [ ]:
df = source_df.copy()
df["profit"] = 1
total_prcnt = 1

for i in range(3,df.shape[0]):
  if df.iloc[i]["open_short"] == 1:
    sell_p_short = df.iloc[i]["close"]
  if df.iloc[i]["open_long"] == 1:
    buy_p_long = df.iloc[i]["close"]
  if df.iloc[i]["close_short"] == 1:
    buy_p_short = df.iloc[i]["close"]
  if df.iloc[i]["close_long"] == 1:
    sell_p_long = df.iloc[i]["close"]
  if df.iloc[i]["close_short"] == 1:
    profit = ((sell_p_short - buy_p_short)/buy_p_short) * 100
    # print(f"Short position ended with profit {profit} % on index {i}") # print info about profit on short order
    total_prcnt = total_prcnt + (total_prcnt * (profit/100))
    # df['profit'][i] = total_prcnt
  if df.iloc[i]["close_long"] == 1:
    profit = ((sell_p_long - buy_p_long)/buy_p_long) * 100
    # print(f"Long position ended with profit {profit} % on index {i}") # print info about profit on long order
    total_prcnt = total_prcnt + (total_prcnt * (profit/100))
    # df['profit'][i] = total_prcnt
  df['profit'][i] = total_prcnt


In [ ]:
df.tail()

In [ ]:
df.loc[df['profit'] == df.profit.max()]

In [ ]:
plt.rcParams["figure.figsize"] = (20,3)
plt.plot(df.index,StandardScaler().fit_transform(pd.DataFrame(df["profit"])), label="Profit", color='blue')
y_line_base_profit = StandardScaler().fit_transform(pd.DataFrame(df["profit"]))[0][0]
plt.axhline(y_line_base_profit,color='blue')
plt.plot(df.index, StandardScaler().fit_transform(pd.DataFrame(df["close"])), label="Close price", color = 'gold')
plt.legend()
plt.show()

In [ ]:
df.tail()